# Flights 

Import libraries

In [139]:
import requests
import json
import pandas as pd
import time
import numpy as np
import pymysql
from datetime import date
from datetime import timedelta
import datetime

setting up variables

In [144]:
key = API_key
today = (date.today()).strftime("%Y-%m-%d")
tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")

In [132]:


airport_list = ['EDDB','EDWR','EDDW','EDDK','EDLW','EDDC','EDDL','EDWE','EDDE','EDNY','EDDH','EDHI','EDAH','EDVK','EDRK','EDDP','EDFM','EDJA','EDDM','EDWO','EDDN','EDLP','EDLR','EDCX','ETNL','EDDR','EDDS','EDLV']

time_1 = f"{today}T00:00"
time_2 = f"{today}T12:00"


In [128]:
def calling_aerodatabox(airport_list, time_1, time_2,key):
    df_all_flights = pd.DataFrame()
    for airport_id in airport_list:
        
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_id}/{time_1}/{time_2}"
        querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
        headers = {"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com","X-RapidAPI-Key": key}

        response = requests.request("GET", url, headers=headers, params=querystring)
        
        if response.status_code == 200:
                 
            df_inc_flights = pd.json_normalize(response.json()['arrivals'])
            
            df_flights_airport = df_inc_flights.filter(['status','departure.airport.name','arrival.scheduledTimeLocal','arrival.terminal','aircraft.model']).assign(icao_code = airport_id)
        
            df_all_flights = pd.concat([df_all_flights, df_flights_airport], ignore_index=True)
                        
        time.sleep(10)
    
    df_all_flights.rename(columns={'departure.airport.name': 'origin','arrival.scheduledTimeLocal':'arrival_time','arrival.terminal': 'terminal', 'aircraft.model' : 'aircraft' }, inplace=True)
    df_all_flights.fillna({'terminal':'None', 'aircraft': 'None'},inplace=True)
        
        
    return df_all_flights


In [129]:
df_all_flights_final = calling_aerodatabox(airport_list, time_1, time_2,key)

In [133]:
df_all_flights_final.head()

,status,origin,arrival_time,terminal,aircraft,icao_code
0,Arrived,Cologne,2022-04-07 05:37+02:00,None,Boeing 767,EDDB
1,Arrived,Unknown,2022-04-07 07:06+02:00,None,Boeing 757-200,EDDB
2,Arrived,Duesseldorf,2022-04-07 07:19+02:00,None,Airbus A319,EDDB
3,Approaching,Cologne,2022-04-07 07:25+02:00,1,Boeing 737-800,EDDB
4,Unknown,Duesseldorf,2022-04-07 07:35+02:00,1,Airbus A319,EDDB


# Upload data

setting connection to cload db

In [136]:
user='username'
password='userpassword'
host='endpoint'
database='databasename'
port='port'

In [ ]:
con= f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

In [138]:
df_all_flights_final.replace({np.nan},'unknown').assign(arrival_time = lambda x: pd.to_datetime(x['arrival_time'])).to_sql('flights', if_exists='append', con=con, index=False)